In [1]:
import mlflow
from utils.evaluate import compare_result, calculate_diff
mlflow.set_tracking_uri("/Users/Pongpanod.Sa/Desktop/cu/thesis/coding/pun_master_thesis/mlruns")

In [2]:
# query_list.append({"augment_name":"window_warp", "augment_params":{"window_ratio": 0.9}})
# query_list.append({"augment_name":"wdba", "augment_params":{"batch_size": 2}})
# query_list = [
#     {"augment_name":"window_warp_wdba", "augment_params":{'window_ratio': 0.1, 'batch_size': 6}},
#     {"augment_name":"window_warp", "augment_params":{"window_ratio": 0.1}},
#     {"augment_name":"wdba", "augment_params":{"batch_size": 6}},
# ]

query_list = []
query_list.append({"augment_name":"wdba", "augment_params":{"batch_size": 9}})


df_hc = compare_result(query_list, ['state'], 0.0)
df_hc

['HandOutlines']


,Adiac,ArrowHead,Beef,BeetleFly,BirdChicken,CBF,Car,ChlorineConcentration,CinCECGTorso,Coffee,Computers,CricketX,CricketY,CricketZ,DiatomSizeReduction,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,DistalPhalanxTW,ECG200,ECG5000,ECGFiveDays,Earthquakes,ElectricDevices,FaceAll,FaceFour,FacesUCR,FiftyWords,Fish,FordA,FordB,GunPoint,Ham,HandOutlines,Haptics,Herring,InlineSkate,InsectWingbeatSound,ItalyPowerDemand,LargeKitchenAppliances,Lightning2,Lightning7,Mallat,Meat,MedicalImages,MiddlePhalanxOutlineAgeGroup,MiddlePhalanxOutlineCorrect,MiddlePhalanxTW,MoteStrain,NonInvasiveFetalECGThorax1,NonInvasiveFetalECGThorax2,OSULeaf,OliveOil,PhalangesOutlinesCorrect,Phoneme,Plane,ProximalPhalanxOutlineAgeGroup,ProximalPhalanxOutlineCorrect,ProximalPhalanxTW,RefrigerationDevices,ScreenType,ShapeletSim,ShapesAll,SmallKitchenAppliances,SonyAIBORobotSurface1,SonyAIBORobotSurface2,StarLightCurves,Strawberry,SwedishLeaf,Symbols,SyntheticControl,ToeSegmentation1,ToeSegmentation2,Trace,TwoLeadECG,TwoPatterns,UWaveGestureLibraryAll,UWaveGestureLibraryX,UWaveGestureLibraryY,UWaveGestureLibraryZ,Wafer,Wine,WordSynonyms,Worms,WormsTwoClass,Yoga
{'batch_size': 9},0.788000,0.726000,0.400000,1.000000,0.650000,1.000000,0.867000,0.700000,0.641000,1.000000,0.748000,0.831000,0.862000,0.815000,0.667000,0.842000,0.822000,0.741000,0.940000,0.961000,1.000000,0.755000,0.772000,0.795000,1.000000,0.964000,0.888000,0.983000,0.939000,0.794000,0.980000,0.838000,0.000000,0.558000,0.500000,0.380000,0.733000,0.991000,0.952000,0.803000,0.822000,0.852000,1.000000,0.825000,0.604000,0.893000,0.682000,0.984000,0.951000,0.958000,0.946000,1.000000,0.872000,0.267000,1.000000,0.937000,0.973000,0.902000,0.512000,0.621000,0.528000,0.840000,0.904000,0.735000,0.967000,0.992000,0.995000,0.965000,0.981000,1.000000,0.908000,0.985000,1.000000,0.917000,0.995000,0.993000,0.925000,0.930000,0.886000,0.998000,0.944000,0.611000,0.792000,0.948000,0.860000
HC2,0.813299,0.862857,0.866667,0.950000,0.900000,1.000000,0.916667,0.771354,1.000000,1.000000,0.764000,0.825641,0.851282,0.864103,0.957516,0.748201,0.775362,0.705036,0.890000,0.947111,1.000000,0.748201,0.757619,0.868047,1.000000,0.963415,0.832967,0.988571,0.953788,0.830864,1.000000,0.695238,0.937838,0.545455,0.671875,0.552727,0.662626,0.971817,0.906667,0.786885,0.821918,0.977825,0.933333,0.807895,0.597403,0.852234,0.558442,0.970447,0.947074,0.967430,0.962810,0.933333,0.831002,0.355485,1.000000,0.853659,0.900344,0.829268,0.538667,0.578667,1.000000,0.921667,0.837333,0.908486,0.929696,0.982152,0.975676,0.964800,0.975879,1.000000,0.964912,0.938462,1.000000,0.999122,1.000000,0.974316,0.854551,0.774707,0.797320,1.000000,0.870370,0.752351,0.753247,0.792208,0.934000


In [3]:
df_hc_result = calculate_diff(df_hc.data)
df_hc_result

Number of columns where {'batch_size': 9} is greater than HC2: 46
Number of columns where {'batch_size': 9} and  HC2 are equal: 7
Number of columns where {'batch_size': 9} is less than  HC2: 31

Columns where {'batch_size': 9} is greater than  HC2: ['BeetleFly', 'CricketX', 'CricketY', 'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect', 'DistalPhalanxTW', 'ECG200', 'ECG5000', 'Earthquakes', 'ElectricDevices', 'FacesUCR', 'FiftyWords', 'Ham', 'Haptics', 'InsectWingbeatSound', 'ItalyPowerDemand', 'LargeKitchenAppliances', 'Lightning2', 'Lightning7', 'Meat', 'MedicalImages', 'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect', 'MiddlePhalanxTW', 'MoteStrain', 'NonInvasiveFetalECGThorax1', 'OliveOil', 'PhalangesOutlinesCorrect', 'ProximalPhalanxOutlineAgeGroup', 'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxTW', 'ScreenType', 'SmallKitchenAppliances', 'SonyAIBORobotSurface2', 'StarLightCurves', 'Strawberry', 'SwedishLeaf', 'Symbols', 'ToeSegmentation2', 'UWaveGestur

/Users/Pongpanod.Sa/Desktop/cu/thesis/coding/pun_master_thesis/utils/evaluate.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[4] = diff_percentages
/Users/Pongpanod.Sa/Desktop/cu/thesis/coding/pun_master_thesis/utils/evaluate.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[5] = compare_results
/Users/Pongpanod.Sa/Desktop/cu/thesis/coding/pun_master_thesis/utils/evaluate.py:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,Adiac,ArrowHead,Beef,BeetleFly,BirdChicken,CBF,Car,ChlorineConcentration,CinCECGTorso,Coffee,...,UWaveGestureLibraryAll,UWaveGestureLibraryX,UWaveGestureLibraryY,UWaveGestureLibraryZ,Wafer,Wine,WordSynonyms,Worms,WormsTwoClass,Yoga
{'batch_size': 9},0.788,0.726,0.4,1.0,0.65,1.0,0.867,0.7,0.641,1.0,...,0.993,0.925,0.93,0.886,0.998,0.944,0.611,0.792,0.948,0.86
HC2,0.813299,0.862857,0.866667,0.95,0.9,1.0,0.916667,0.771354,1.0,1.0,...,0.974316,0.854551,0.774707,0.79732,1.0,0.87037,0.752351,0.753247,0.792208,0.934
Percentage Diff,3.210563,18.850846,116.666667,5.263158,38.461538,0.0,5.728566,10.193452,56.00624,0.0,...,1.91765,8.244038,20.045405,11.122269,0.200401,8.459575,23.134386,5.144828,19.665574,8.604651
Result,Lose,Lose,Lose,Win,Lose,Tie,Lose,Lose,Lose,Tie,...,Win,Win,Win,Win,Lose,Win,Lose,Win,Win,Lose


In [ ]:
df_hc_result.to_csv('xx.csv')

In [15]:
query_list = []
query_list.append({"augment_name":"wdba", "augment_params":{"batch_size": 9}})


df_bl = compare_result(query_list, ['baseline'], 0.0)
df_bl

['HandOutlines']


,Adiac,ArrowHead,Beef,BeetleFly,BirdChicken,CBF,Car,ChlorineConcentration,CinCECGTorso,Coffee,Computers,CricketX,CricketY,CricketZ,DiatomSizeReduction,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,DistalPhalanxTW,ECG200,ECG5000,ECGFiveDays,Earthquakes,ElectricDevices,FaceAll,FaceFour,FacesUCR,FiftyWords,Fish,FordA,FordB,GunPoint,Ham,HandOutlines,Haptics,Herring,InlineSkate,InsectWingbeatSound,ItalyPowerDemand,LargeKitchenAppliances,Lightning2,Lightning7,Mallat,Meat,MedicalImages,MiddlePhalanxOutlineAgeGroup,MiddlePhalanxOutlineCorrect,MiddlePhalanxTW,MoteStrain,NonInvasiveFetalECGThorax1,NonInvasiveFetalECGThorax2,OSULeaf,OliveOil,PhalangesOutlinesCorrect,Phoneme,Plane,ProximalPhalanxOutlineAgeGroup,ProximalPhalanxOutlineCorrect,ProximalPhalanxTW,RefrigerationDevices,ScreenType,ShapeletSim,ShapesAll,SmallKitchenAppliances,SonyAIBORobotSurface1,SonyAIBORobotSurface2,StarLightCurves,Strawberry,SwedishLeaf,Symbols,SyntheticControl,ToeSegmentation1,ToeSegmentation2,Trace,TwoLeadECG,TwoPatterns,UWaveGestureLibraryAll,UWaveGestureLibraryX,UWaveGestureLibraryY,UWaveGestureLibraryZ,Wafer,Wine,WordSynonyms,Worms,WormsTwoClass,Yoga
{'batch_size': 9},0.788000,0.726000,0.400000,1.000000,0.650000,1.000000,0.867000,0.700000,0.641000,1.000000,0.748000,0.831000,0.862000,0.815000,0.667000,0.842000,0.822000,0.741000,0.940000,0.961000,1.000000,0.755000,0.772000,0.795000,1.000000,0.964000,0.888000,0.983000,0.939000,0.794000,0.980000,0.838000,0.000000,0.558000,0.500000,0.380000,0.733000,0.991000,0.952000,0.803000,0.822000,0.852000,1.000000,0.825000,0.604000,0.893000,0.682000,0.984000,0.951000,0.958000,0.946000,1.000000,0.872000,0.267000,1.000000,0.937000,0.973000,0.902000,0.512000,0.621000,0.528000,0.840000,0.904000,0.735000,0.967000,0.992000,0.995000,0.965000,0.981000,1.000000,0.908000,0.985000,1.000000,0.917000,0.995000,0.993000,0.925000,0.930000,0.886000,0.998000,0.944000,0.611000,0.792000,0.948000,0.860000
baseline,0.823529,0.857143,0.833333,0.900000,0.900000,0.998889,0.916667,0.749479,0.865217,1.000000,0.720000,0.800000,0.815385,0.828205,0.924837,0.755396,0.771739,0.683453,0.920000,0.944667,1.000000,0.755396,0.746725,0.804142,0.988636,0.956585,0.839560,0.977143,0.949242,0.806173,0.993333,0.714286,0.943243,0.542208,0.703125,0.467273,0.669697,0.962099,0.861333,0.754098,0.808219,0.942431,0.950000,0.798684,0.590909,0.835052,0.538961,0.928914,0.948601,0.967430,0.954545,0.933333,0.831002,0.292194,1.000000,0.848780,0.903780,0.800000,0.504000,0.480000,1.000000,0.926667,0.834667,0.891847,0.923400,0.982394,0.981081,0.966400,0.983920,0.983333,0.960526,0.907692,1.000000,0.998244,0.997250,0.972641,0.853155,0.772194,0.798157,0.999027,0.870370,0.753918,0.753247,0.792208,0.915000


In [16]:
df_bl_result = calculate_diff(df_bl.data)
df_bl_result

Number of columns where {'batch_size': 9} is greater than baseline: 49
Number of columns where {'batch_size': 9} and  baseline are equal: 4
Number of columns where {'batch_size': 9} is less than  baseline: 31

Columns where {'batch_size': 9} is greater than  baseline: ['BeetleFly', 'CBF', 'Computers', 'CricketX', 'CricketY', 'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect', 'DistalPhalanxTW', 'ECG200', 'ECG5000', 'ElectricDevices', 'FaceFour', 'FacesUCR', 'FiftyWords', 'Fish', 'Ham', 'Haptics', 'InsectWingbeatSound', 'ItalyPowerDemand', 'LargeKitchenAppliances', 'Lightning2', 'Lightning7', 'Meat', 'MedicalImages', 'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect', 'MiddlePhalanxTW', 'MoteStrain', 'NonInvasiveFetalECGThorax1', 'OliveOil', 'PhalangesOutlinesCorrect', 'ProximalPhalanxOutlineAgeGroup', 'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxTW', 'RefrigerationDevices', 'ScreenType', 'SmallKitchenAppliances', 'SonyAIBORobotSurface2', 'StarLightCurves', 'St

/var/folders/5h/wm_jzw990p97r0nbgrywjmt00000gp/T/ipykernel_16295/70986375.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[4] = diff_percentages
/var/folders/5h/wm_jzw990p97r0nbgrywjmt00000gp/T/ipykernel_16295/70986375.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[5] = compare_results


,Adiac,ArrowHead,Beef,BeetleFly,BirdChicken,CBF,Car,ChlorineConcentration,CinCECGTorso,Coffee,...,UWaveGestureLibraryAll,UWaveGestureLibraryX,UWaveGestureLibraryY,UWaveGestureLibraryZ,Wafer,Wine,WordSynonyms,Worms,WormsTwoClass,Yoga
{'batch_size': 9},0.788,0.726,0.4,1.0,0.65,1.0,0.867,0.7,0.641,1.0,...,0.993,0.925,0.93,0.886,0.998,0.944,0.611,0.792,0.948,0.86
baseline,0.823529,0.857143,0.833333,0.9,0.9,0.998889,0.916667,0.749479,0.865217,1.0,...,0.972641,0.853155,0.772194,0.798157,0.999027,0.87037,0.753918,0.753247,0.792208,0.915
4,4.508809,18.063754,108.333333,11.111111,38.461538,0.111235,5.728566,7.068452,34.979312,0.0,...,2.093169,8.421139,20.436009,11.005666,0.102866,8.459574,23.390916,5.144828,19.665574,6.395349
5,Lose,Lose,Lose,Win,Lose,Win,Lose,Lose,Lose,Tie,...,Win,Win,Win,Win,Lose,Win,Lose,Win,Win,Lose


--------

--------

In [57]:
df_data_no_0 = df_data_no_0.T
df_data_no_0.rename(columns = {3:'Percentage Diff', 4:'Result'}, inplace=True)

df_data_no_0

,{'batch_size': 9},baseline,Percentage Diff,Result
Adiac,0.788,0.823529,4.508809,Lose
ArrowHead,0.726,0.857143,18.063754,Lose
Beef,0.4,0.833333,108.333333,Lose
BeetleFly,1.0,0.9,11.111111,Win
BirdChicken,0.65,0.9,38.461538,Lose
...,...,...,...,...
Wine,0.944,0.87037,8.459574,Win
WordSynonyms,0.611,0.753918,23.390916,Lose
Worms,0.792,0.753247,5.144828,Win
WormsTwoClass,0.948,0.792208,19.665574,Win


In [70]:
df_win = df_data_no_0.query("Result == 'Win'")
df_lose = df_data_no_0.query("Result == 'Lose'")

df_win[['Percentage Diff']] = df_win[['Percentage Diff']].astype(float)
df_lose[['Percentage Diff']] = df_lose[['Percentage Diff']].astype(float)

/var/folders/5h/wm_jzw990p97r0nbgrywjmt00000gp/T/ipykernel_1706/2985193053.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_win[['Percentage Diff']] = df_win[['Percentage Diff']].astype(float)
/var/folders/5h/wm_jzw990p97r0nbgrywjmt00000gp/T/ipykernel_1706/2985193053.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lose[['Percentage Diff']] = df_lose[['Percentage Diff']].astype(float)


In [72]:
df_lose.describe()

,Percentage Diff
count,31.000000
mean,16.593217
std,25.476281
min,0.052408
25%,1.120359
50%,6.395349
75%,22.153112
max,108.333333


In [29]:
df_data_no_0.to_csv('xx.csv')

Number of columns where wdba is greater than baseline: 48
Number of columns where wdba and baseline are equal: 4
Number of columns where wdba is less than baseline: 31

Columns where wdba is greater than baseline: ['BeetleFly', 'CBF', 'Computers', 'CricketX', 'CricketY', 'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect', 'DistalPhalanxTW', 'ECG200', 'ECG5000', 'ElectricDevices', 'FaceFour', 'FacesUCR', 'FiftyWords', 'Fish', 'Ham', 'Haptics', 'InsectWingbeatSound', 'ItalyPowerDemand', 'LargeKitchenAppliances', 'Lightning2', 'Lightning7', 'Meat', 'MedicalImages', 'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect', 'MiddlePhalanxTW', 'MoteStrain', 'NonInvasiveFetalECGThorax1', 'OliveOil', 'PhalangesOutlinesCorrect', 'ProximalPhalanxOutlineAgeGroup', 'ProximalPhalanxOutlineCorrect', 'ProximalPhalanxTW', 'RefrigerationDevices', 'ScreenType', 'SmallKitchenAppliances', 'SonyAIBORobotSurface2', 'StarLightCurves', 'Strawberry', 'SyntheticControl', 'ToeSegmentation2', 'UWa

In [35]:
df_data_no_0.T.describe()

,{'batch_size': 9},baseline,3
count,83.000000,83.000000,83.000000
mean,0.842120,0.848205,10.086514
std,0.165061,0.144245,16.951458
min,0.267000,0.292194,0.000000
25%,0.763500,0.795183,1.475918
50%,0.888000,0.870370,5.716981
75%,0.976500,0.958556,10.355569
max,1.000000,1.000000,108.333333


In [33]:
query_list = []

for i in range(1, 6):
    query_list.append({"augment_name":"discriminative_guided_warp", "augment_params":{"batch_size": i}})

df = compare_result(query_list)
df


KeyboardInterrupt: 

In [ ]:
len(['Symbols', 'WormsTwoClass', 'UWaveGestureLibraryZ', 'UWaveGestureLibraryAll', 'Wine', 'UWaveGestureLibraryX', 'Mallat', 'SwedishLeaf', 'Strawberry', 'CinCECGTorso', 'HandOutlines', 'SyntheticControl', 'UWaveGestureLibraryY', 'Wafer', 'StarLightCurves', 'ElectricDevices'])

16